In [14]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [15]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [16]:
df = pd.read_csv("../data/lupus.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'lupus'

In [17]:
pd.value_counts(y)

0    52
1    35
Name: count, dtype: int64

In [18]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [59]:
def fit_trees():
    learning_rate=10
    max_depth=3
    bins=5
    lam=10

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 1
    verbose = 1
    tolerance= 0.01

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 1
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

with open('./benchmark/'+dataset+'.csv','w') as f:
        f.writelines("")

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    # train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    # test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    
    # y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    # train_f1 = mean_squared_error(y_train_one_hot.toarray(),gtgp.train_p)
    # test_f1 = mean_squared_error(y_test_one_hot.toarray(),gtgp.test_p)


    train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    test_f1 = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)

    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),gtgp.train_p)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),gtgp.test_p)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

7 21 1
9 27 2
9 27 3
10 30 4
10 30 5
11 33 6
11 33 7
11 33 8
11 33 9
11 33 10
11 33 11
11 33 12
11 33 13
11 33 14
11 33 15
11 33 16
11 33 17
11 33 18
11 33 19
11 33 20
11 33 21
11 33 22
11 33 23
11 33 24
11 33 25
11 33 26
11 33 27
11 33 28
11 33 29
11 33 30
11 33 31
11 33 32
11 33 33
11 33 34
11 33 35
11 33 36
11 33 37
11 33 38
11 33 39
11 33 40
11 33 41
11 33 42
11 33 43
11 33 44
11 33 45
11 33 46
11 33 47
11 33 48
11 33 49
11 33 50
11 33 51
11 33 52
11 33 53
11 33 54
11 33 55
11 33 56
11 33 57
11 33 58
11 33 59
11 33 60
11 33 61
11 33 62
11 33 63
11 33 64
11 33 65
11 33 66
11 33 67
11 33 68
11 33 69
11 33 70
11 33 71
11 33 72
11 33 73
11 33 74
11 33 75
11 33 76
11 33 77
11 33 78
11 33 79
11 33 80
11 33 81
11 33 82
11 33 83
11 33 84
11 33 85
11 33 86
11 33 87
11 33 88
11 33 89
11 33 90
11 33 91
11 33 92
11 33 93
11 33 94
11 33 95
11 33 96
11 33 97
11 33 98
11 33 99
11 33 100
retrain  1 :
	train: 0.9166666666666666 13.766281461843306 	test: 0.7407407407407407 9.31599943984584
         

# Decision Tree

In [62]:
from sklearn.tree import DecisionTreeClassifier

In [63]:
with open('./benchmark_DC/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        clf = DecisionTreeClassifier()
        clf.fit(X_train,y_train)
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

        num_trees = 1
        depth = clf.tree_.max_depth
        num_nodes = clf.tree_.node_count

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [61]:
import xgboost

In [8]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier(n_estimators=10)
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        # train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier(n_estimators=10)
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        # train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))


        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # rfc = RandomForestClassifier(n_estimators=100)
        rfc = RandomForestClassifier(n_estimators=100)
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        # train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')
        
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)